In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=f6868d8945524a31f5c20397af4aecbe5a2ca8f66c040f6a773f160ca2d7ba03
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import HiveContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

import re
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np
import string

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext

from pyspark import HiveContext

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext=HiveContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:720: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
df1 =sqlContext.read.format('com.databricks.spark.csv')\
.options(header='true',inferschema='true')\
.load("/content/Reviews.csv")

In [ ]:
df1.count()

568454

In [ ]:
df1.show()

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|B000UA0QIQ|A395BORC6FGVXV|                Karl|            

In [ ]:
df=df1

#### Mengecek dan menghapus missing value

In [ ]:
variable_name="Text"
# Menghitung jumlah missing value pada variabel tertentu
missing_count = df.select(variable_name) \
    .where(col(variable_name).isNull() | isnan(variable_name)) \
    .count()

# Menampilkan jumlah missing value
print("Jumlah missing value pada variabel", variable_name, "adalah", missing_count)

Jumlah missing value pada variabel Text adalah 10


In [ ]:
df = df.dropna()
df.count()

568444

#### Mengecek tipe datanya. kalo masi string, diubah dulu ke numerik

In [ ]:
#mengecek tipe data
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
# Mengubah tipe data kolom "score" menjadi integer
df = df.withColumn("Score", col("Score").cast(IntegerType()))


#### Menentukan label sentimen berdasarkan score nya

In [ ]:
import pyspark.sql.functions as f

df=df.filter((f.col('Score')!=3))
df.count()

525660

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Definisikan fungsi untuk menentukan label sentimen
def get_sentiment_label(score):
    if score > 3:
        return "positif"
    elif score < 3:
        return "negatif"
    else:
        return "netral"

# Daftarkan fungsi sebagai User Defined Function (UDF) pada PySpark
sentiment_udf = udf(get_sentiment_label, StringType())

# Menerapkan UDF pada kolom "score" dalam DataFrame
df = df.withColumn("sentiment", sentiment_udf("Score"))
df.show(4)

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|sentiment|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|  positif|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|  negatif|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|  positif|
|  4

#### kolom sentimen, dijadiin biner 0 dan 1

In [ ]:
def toBinary(score):
    if score >= 3: return 1
    else: return 0
udfScoretoBinary=udf(toBinary, StringType())

df = df.withColumn("Target", udfScoretoBinary("Score"))
print(df.show(4))
print(df.count())


+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|sentiment|Target|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|  positif|     1|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|  negatif|     0|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"

#### Preprocessing

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#TEXT Pre-processing

##COnvert to lower
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def lower(text):
    return text.lower()

lower_udf =udf(lower,StringType())


##Remove nonAscii
def strip_non_ascii(data_str):
#''' Returns the string without non ASCII characters'''
    stripped = (c for c in data_str if 0 < ord(c) < 127)
    return ''.join(stripped)
# setup pyspark udf function
strip_non_ascii_udf = udf(strip_non_ascii, StringType())

##FIx abbreviations
def fix_abbreviation(data_str):
    data_str = data_str.lower()
    data_str = re.sub(r'\bthats\b', 'that is', data_str)
    data_str = re.sub(r'\bive\b', 'i have', data_str)
    data_str = re.sub(r'\bim\b', 'i am', data_str)
    data_str = re.sub(r'\bya\b', 'yeah', data_str)
    data_str = re.sub(r'\bcant\b', 'can not', data_str)
    data_str = re.sub(r'\bdont\b', 'do not', data_str)
    data_str = re.sub(r'\bwont\b', 'will not', data_str)
    data_str = re.sub(r'\bid\b', 'i would', data_str)
    data_str = re.sub(r'wtf', 'what the fuck', data_str)
    data_str = re.sub(r'\bwth\b', 'what the hell', data_str)
    data_str = re.sub(r'\br\b', 'are', data_str)
    data_str = re.sub(r'\bu\b', 'you', data_str)
    data_str = re.sub(r'\bk\b', 'OK', data_str)
    data_str = re.sub(r'\bsux\b', 'sucks', data_str)
    data_str = re.sub(r'\bno+\b', 'no', data_str)
    data_str = re.sub(r'\bcoo+\b', 'cool', data_str)
    data_str = re.sub(r'rt\b', '', data_str)
    data_str = data_str.strip()
    return data_str

##Remove punctuations mentions and alphanumeric characters
def remove_features(data_str):
# compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
# convert to lowercase
    data_str = data_str.lower()
# remove hyperlinks
    data_str = url_re.sub(' ', data_str)
# remove @mentions
    data_str = mention_re.sub(' ', data_str)
# remove puncuation
    data_str = punc_re.sub(' ', data_str)
# remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
# remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word):
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word):
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
# remove unwanted space, *.split() will automatically split on
# whitespace and discard duplicates, the " ".join() joins the
# resulting list into one string.
    return " ".join(cleaned_str.split())
# setup pyspark udf function

##Remove stop words
def stopwords_delete(word_list):
        from nltk.corpus import stopwords
        filtered_words=[]
        return word_list

# Part-of-Speech Tagging
def tag_and_remove(data_str):
    cleaned_str = ' '
# noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
# adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
# verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
# break string into 'words'
    text = data_str.split()
# tag the text and keep only those with the right tags
    tagged_text = pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '
    return cleaned_str

##Lemmatization
def lemmatize(data_str):
# expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    tagged_words = pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str


In [ ]:
lower_udf =udf(lower,StringType())
strip_non_ascii_udf = udf(strip_non_ascii, StringType())
fix_abbreviation_udf = udf(fix_abbreviation, StringType())
remove_features_udf = udf(remove_features, StringType())
remove_stops_udf = udf(stopwords_delete, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())

In [ ]:
df = df.withColumn("lower_text",lower_udf(df["Text"]))
df = df.withColumn("text_non_asci",fix_abbreviation_udf(df["lower_text"]))
df = df.withColumn("fixed_abbrev",fix_abbreviation_udf(df["text_non_asci"]))
df = df.withColumn('removed_features',remove_features_udf(df['fixed_abbrev']))
df.show(5,True)

+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+------+--------------------+--------------------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|sentiment|Target|          lower_text|       text_non_asci|        fixed_abbrev|    removed_features|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+---------+------+--------------------+--------------------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|  positif|     1|i have bought sev...|i have bought sev...|i have bought s

In [ ]:
df_no_stop_words = df.withColumn("stopwords_delete", remove_stops_udf("removed_features")).select('Text','stopwords_delete','Target')
df_no_stop_words.show(5)

+--------------------+--------------------+------+
|                Text|    stopwords_delete|Target|
+--------------------+--------------------+------+
|I have bought sev...|i have bought sev...|     1|
|"Product arrived ...|product arrived l...|     0|
|"This is a confec...|this is a confect...|     1|
|If you are lookin...|if you are lookin...|     0|
|Great taffy at a ...|great taffy at a ...|     1|
+--------------------+--------------------+------+
only showing top 5 rows



In [ ]:
df_pos_tagging=df_no_stop_words.withColumn("tag_and_remove_pos", tag_and_remove_udf("stopwords_delete")).select('Text','tag_and_remove_pos','Target')

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#Tokenizing the document
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="tag_and_remove_pos", outputCol="words")
wordsDataFrame = tokenizer.transform(df_pos_tagging)
for words_label in wordsDataFrame.select("words", "Target").take(3):
    print(words_label)

df_text = df.withColumn("text_lower",lower_udf(df["Text"])).select('text_lower','Target')

Row(words=['', 'i', 'have', 'bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'have', 'found', 'be', 'good', 'quality', 'product', 'looks', 'stew', 'processed', 'meat', 'smells', 'labrador', 'is', 'finicky', 'appreciates', 'product', 'better', 'most'], Target='1')
Row(words=['', 'product', 'arrived', 'labeled', 'jumbo', 'salted', 'peanuts', 'peanuts', 'were', 'small', 'sized', 'unsalted', 'sure', 'was', 'error', 'vendor', 'intended', 'represent', 'product', 'jumbo'], Target='0')
Row(words=['', 'is', 'confection', 'has', 'been', 'few', 'centuries', 'is', 'light', 'pillowy', 'citrus', 'gelatin', 'nuts', 'case', 'filberts', 'is', 'cut', 'tiny', 'squares', 'coated', 'powdered', 'sugar', 'is', 'tiny', 'mouthful', 'heaven', 'chewy', 'flavorful', 'i', 'recommend', 'yummy', 'treat', 'are', 'familiar', 'story', 'c', 's', 'lewis', 'lion'], Target='1')


In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="words_filtered")
wordsDataFrame1 = remover.transform(wordsDataFrame).select("Target","words_filtered")
wordsDataFrame1.show(2)

+------+--------------------+
|Target|      words_filtered|
+------+--------------------+
|     1|[, bought, severa...|
|     0|[, product, arriv...|
+------+--------------------+
only showing top 2 rows



In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df_text_lemma=df_pos_tagging.withColumn("lemmatized_text",lemmatize_udf("tag_and_remove_pos")).select('Text','lemmatized_text','Target')
df_text_lemma.show(5)

+--------------------+--------------------+------+
|                Text|     lemmatized_text|Target|
+--------------------+--------------------+------+
|I have bought sev...|i have buy severa...|     1|
|"Product arrived ...|product arrive la...|     0|
|"This is a confec...|be confection hav...|     1|
|If you are lookin...|be look secret in...|     0|
|Great taffy at a ...|great taffy great...|     1|
+--------------------+--------------------+------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID
df_text_lemma = df_text_lemma.withColumn("uid", monotonically_increasing_id())
df_text_lemma.show(4)

+--------------------+--------------------+------+---+
|                Text|     lemmatized_text|Target|uid|
+--------------------+--------------------+------+---+
|I have bought sev...|i have buy severa...|     1|  0|
|"Product arrived ...|product arrive la...|     0|  1|
|"This is a confec...|be confection hav...|     1|  2|
|If you are lookin...|be look secret in...|     0|  3|
+--------------------+--------------------+------+---+
only showing top 4 rows



### Preparing data for modelling

In [ ]:
data = df_text_lemma.select('uid', 'lemmatized_text','Target')
#data=wordsDataFrame2
data.show(4)

+---+--------------------+------+
|uid|     lemmatized_text|Target|
+---+--------------------+------+
|  0|i have buy severa...|     1|
|  1|product arrive la...|     0|
|  2|be confection hav...|     1|
|  3|be look secret in...|     0|
+---+--------------------+------+
only showing top 4 rows



####Membagi data training dan testing

In [ ]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
# Caching the RDD for training
trainingData
#Renaming features for modeling
training = trainingData.selectExpr("lemmatized_text as text", "Target as label")
training = training.withColumn("label", training["label"].cast(DoubleType()))

In [ ]:
# Caching the RDD for test
testData
#Renaming features for modeling
test = testData.selectExpr("lemmatized_text as text", "Target as label")
test = test.withColumn("label", test["label"].cast(DoubleType()))

### Tokenizing the Training set anc creating TF-IDF matrix using HashingTF

In [ ]:
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="hashing")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

### Modelling dan Evaluasi

### Logistic Regression Model

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
import time

start_time = time.time()
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])
# Training the model
model = pipeline.fit(training)
end_time = time.time()

training_time = end_time - start_time
print("Waktu pelatihan model: ", training_time, " detik")

Waktu pelatihan model:  6021.695109605789  detik


In [ ]:
#Predicing Output
prediction = model.transform(test)

In [ ]:
prediction.select("label", "prediction").show(10,False)

+-----+----------+
|label|prediction|
+-----+----------+
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
|1.0  |1.0       |
+-----+----------+
only showing top 10 rows



In [ ]:
prediction.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hashing: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(prediction)

0.9152259959378706

### Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
pipeline_nb = Pipeline(stages=[tokenizer, hashingTF, idf, nb])
# Training the model
model_nb = pipeline_nb.fit(training)

In [ ]:
#Predicing Output
prediction_nb = model_nb.transform(test)

In [ ]:
prediction_nb.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hashing: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_nb = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_nb.evaluate(prediction_nb)

0.8783411775648421